### ngram / file 가져오는 함수

In [1]:
from os import listdir
def fileids(path):
    return [path+file for file in listdir(path)]

In [2]:
def ngram(term, n=2):
    return [term[i:i+n] for i in range(len(term) - n + 1)]

### Tokenizing

In [3]:
from nltk.tokenize import word_tokenize
from konlpy.tag import Komoran

ma = Komoran()

# 여러가지 토큰 방법 일단은
def tokenize(file):
    with open(file, encoding='utf-8') as fp:
        content = fp.read()
    tokens1 = content.split() # 원시어절
    tokens2 = word_tokenize(content) # 구두점 분리 => 원시어절
    tokens3 = [_ for token in tokens2 for _ in ma.pos(token)] # 형태소-품사
    tokens4 = [token[0] for token in tokens3] # 형태소
    tokens5 = [token[0] for token in tokens3 if token[1].startswith('N')] # 명사
    tokens6 = [_ for token in tokens4 for _ in ngram(token)] # ngram

    print("..")
    print(len(tokens1))
    print(len(tokens1 + tokens2 + tokens3))
    print(len(tokens1 + tokens2 + tokens3 + tokens4 + tokens5 + tokens6))
    print(len(set(tokens1 + tokens2 + tokens3 + tokens4 + tokens5 + tokens6)))
    
    return tokens1
# token을 어떤걸 써야하나

## 구두점 및 불용어 등등 처리

In [65]:
def filecontent(file):
    with open(file, encoding='utf-8') as fp:
        content = fp.read()
    return content

In [1]:
from string import punctuation
import re

def makePattern():
    pattern = dict()

    # 구두점
    pattern1 = re.compile(r'[{0}]'.format(re.escape(punctuation)))
    pattern['punc'] = pattern1
    # corpus = pattern1.sub(' ',corpus)

    # 불용어
    pattern2 = re.compile(r'[A-Za-z0-9]{7,}')
    pattern['stop'] = pattern2
    # corpus = pattern2.sub(' ',corpus)

    # 이메일
    # pattern3 = re.compile(r'\w{2,}@\w{3,}(.\w{2,})+')
    pattern3 = re.compile(r'\w{2,}@(.?\w{2,})+')
    pattern['email'] = pattern3
    # corpus = pattern3.sub(' ',corpus)

    # 도메인
    pattern4 = re.compile(r'(.?\w{2,}){2,}')
    pattern['url'] = pattern4
    # corpus = pattern4.sub(' ',corpus)

    # 한글 이외
    pattern5 = re.compile(r'[^가-힣0-9]+')
    pattern['nonkorean'] = pattern5
    # corpus = pattern5.sub(' ',corpus)

    # WhiteSpace
    pattern6 = re.compile(r"\s{2,}")
    pattern['whitespace'] = pattern5
    # corpus = pattern6.sub(' ',corpus)
    
    return pattern

In [78]:
from collections import defaultdict

# content = filecontent(fileids('./news_crawl_project/')[-2])

pattern = makePattern()

def punc_stop(file):
    for _ in ['email', 'punc', 'stop','whitespace']:
        file = pattern[_].sub(' ',file)
    return file

def indexing(file):
    indexTerm1 = defaultdict(int)
    indexTerm2 = defaultdict(int)
    indexTerm3 = defaultdict(int)
    indexTerm4 = defaultdict(int)
    indexTerm5 = defaultdict(int)
    indexTerm6 = defaultdict(int)

    for term in word_tokenize(file):
        indexTerm1[term] += 1 # 원시어절
    
    for _ in indexTerm1:
        for t in ma.pos(_):
            indexTerm2[t] += 1 # 원시형태소+품사
            if len(t[0]) > 1: # 음절 길이로 정규화
                indexTerm3[t[0]] += 1 # 원시형태소
            if t[1].startswith('N'):
                indexTerm4[t[0]] += 1 # 명사
            for n in ngram(t[0]): # 바이그램
                indexTerm5[n] += 1
                indexTerm6[n] += 1
    
    return indexTerm4 #일단은 명사만 

## DocTermMatrix -> TermDocMatrix

In [79]:
def DTM_conversion():
    documentList = defaultdict(lambda: defaultdict(int))
    idx = 0
    for file in fileids('./news_crawl_project/'):
        documentList[idx] = indexing(punc_stop(filecontent(file)))
        idx += 1
        if idx % 100 == 0:
            print(idx, '진행중')
    return documentList
DTM = DTM_conversion()

100 진행중
200 진행중
300 진행중
400 진행중
500 진행중
600 진행중
700 진행중
800 진행중
900 진행중
1000 진행중
1100 진행중
1200 진행중


In [80]:
def TDM_conversion(DTM):
    TDM = defaultdict(lambda:defaultdict(int))
    for idx, termList in DTM.items():
        for term, freq in termList.items():
            TDM[term][idx] = freq
    return TDM
TDM = TDM_conversion(DTM)

## TermDocMatrix -> TermWeightMatrix

In [81]:
from math import sqrt, log2

N = len(DTM)

TWM = defaultdict(lambda:defaultdict(float))
DVL = defaultdict(float)

for idx, termList in DTM.items():
    if len(termList) > 0:
        maxTF = max(termList.values()) 
        
        for term, freq in termList.items():
            TF = freq/maxTF
            IDF = log2(N/len(TDM[term]))
            TWM[term][idx] = TF*IDF
            DVL[idx] += TWM[term][idx]**2
        
for idx, length in DVL.items():
    DVL[idx] = sqrt(length)

## Query에서 색인어 추출

In [83]:
from nltk.tokenize import sent_tokenize

def TQM_conversion(query):
    
    indexTerm1 = defaultdict(int)
    indexTerm2 = defaultdict(int)
    indexTerm3 = defaultdict(int)
    indexTerm4 = defaultdict(int)
    indexTerm5 = defaultdict(int)
    indexTerm6 = defaultdict(int)

    for _ in word_tokenize(query):
        for t in ma.pos(_):
            indexTerm2[t] += 1 # 원시형태소+품사
            if len(t[0]) > 1: # 음절 길이로 정규화
                indexTerm3[t[0]] += 1 # 원시형태소
            if t[1].startswith('N'):
                indexTerm4[t[0]] += 1 # 명사
            for n in ngram(t[0]): # 바이그램
                indexTerm5[n] += 1
                indexTerm6[n] += 1
    return indexTerm4

query = '서울시에 거래되는 아파트의 전세값은?'
TQM = TQM_conversion(query)

In [84]:
TQM

defaultdict(int, {'서울시': 1, '거래': 1, '아파트': 1, '전세': 1, '값': 1})

## 쿼리 가중치 계산

In [73]:
QWM = defaultdict(float)
        
alpha = 0.5
maxTF = max(TQM.values())
for term, ferq in TQM.items():
    TF = alpha + (1-alpha)*(freq/maxTF)
    DF = len(TWM[term]) if len(TWM[term]) > 0 else 1
    IDF = log2(N/DF)
    QWM[term] = TF*IDF

In [74]:
QWM

defaultdict(float,
            {'서울시': 4.646162657157894,
             '거래': 4.042091333489033,
             '아파트': 4.080565481303669,
             '전세': 5.897701424153858,
             '값': 5.335822536545743})

## 유사도 계산 (Euclidean vs. Cosine) -> 검색결과 출력

#### Cosine

In [57]:
candidateList = defaultdict(float)
for term, weight1 in QWM.items():
    for doc, weight2 in TWM[term].items():
        innerProduct = weight1 * weight2
        candidateList[doc] += innerProduct
        
for doc, sim in candidateList.items():
    candidateList[doc] = sim/DVL[doc]
    
K = 5

for doc, sim in sorted(candidateList.items(), key=lambda x:x[1], reverse=True)[:K]:
    print('문서이름:{0} / 유사도:{1:.4f}'.format(doc, sim))
    print(punc_stop(filecontent(fileids('./news_crawl_project/')[doc])))

문서이름:270 / 유사도:1.7061
 기존 매물도 많은데 얼마 깎아야 팔리나 걱정 촛불집회까지 강남은 신도시와 무관 무덤덤 호가 올라 추격 매수는 주춤 서울 연합뉴스 서미숙 홍국기 기자 3기 신도시 발표후 집을 사겠다는 사람은 종적을 감추고 매물을 얼마나 더 싸게 내놔야 팔리겠냐는 집주인들 문의만 옵니다 수천만원 정도는 우습게 빠질 것 같네요 지난 11일 일산서구 후곡마을에 위치한 한 중개업소 대표 말이다 그는 이 곳은 지난해 9 13대책 이후 거래가 끊겨서 나온지 몇 달 된 물건들도 수두룩한데 3기 신도시 소식을 듣고 누가 집을 사겠느냐 며 지난해 5억원 하던 전용 84 아파트값이 최근 4억2천만 4억3천만원으로 내려왔지만 실거래가 되려면 이보다 더 낮춰야 할 것 이라고 말했다 지난 7일 3기 신도시 발표 이후 일산 파주 인천 서구 등 신도시의 직접적인 영향권에 있는 지역은 마치 찬물을 끼얹은 듯 시장 분위기가 냉랭했다 당장 급매물이 추가로 쏟아지거나 가격이 급락하진 않았지만 매수세가 끊기면서 집값 하락을 걱정하는 목소리가 많다 반면 강남을 비롯한 서울은 평온한 분위기 속에 최근 급매물 소진 이후 이어졌던 추격 매수세는 다소 주춤해진 모습이다 연합뉴스 자료사진 가뜩이나 안좋은데 찬물 끼얹은 수도권 외곽 이번 고양 창릉 등 3기 신도시 건설 계획에 가장 크게 반대하고 있는 일산서구 아파트 시장은 아예 매수세가 실종됐다 일산서구 주엽동의 한 중개업소 대표는 고양 원흥 삼송지구 등 인근 새 아파트 입주로 이 일대가 대규모 베드타운이 됐는데 또다시 일산신도시 절반 수준의 신도시가 들어선다고 하니 누가 집을 사겠느냐 며 신도시 발표 후 매수 문의는 한 통도 없고 기존에 매물을 내놨던 집주인들한테 얼마를 더 낮춰야 집이 팔리겠냐고 걱정하는 전화만 온다 고 분위기를 전했다 인터넷 커뮤니티 등에는 일부 사정이 다급한 매도자들이 1천만 2천만원 이상 가격을 추가로 낮춰 내놨다는 글이 올라오기도 했다 일산서구 일산동의 한 중개업소 사장은 이 곳이 2017년 8 2대책에서 

### Euclidean

In [75]:
candidateList = defaultdict(float)
for term, docList in TWM.items():
    for doc, weight1 in docList.items():
        weight2 = QWM[term]
        candidateList[doc] += (weight1 - weight2)**2 
        
for doc, sim in candidateList.items():
    candidateList[doc] = sqrt(sim)

# for doc in DTM:
#     print(doc, len(punc_stop(filecontent(fileids('./news_crawl_project/')[doc])).split()), len(DTM[doc]), sum(DTM[doc].values()))

K = 5
for doc, sim in sorted(candidateList.items(), key=lambda x:x[1])[:K]:
    print('문서이름:{0} / 거리:{1:.4f}'.format(doc, sim))
    print(punc_stop(filecontent(fileids('./news_crawl_project/')[doc])))
    print()

문서이름:115 / 거리:4.0010
 국내에서 유튜브를 가장 많이 보는 연령대는 50대 이상 인 것으로 나타났다 50대 이상 유튜브의 사용시간은 1년 사이 두 배로 늘었다 앱 분석업체 와이즈앱이 지난 4월 전국 안드로이드 스마트폰 사용자 3만3000명을 조사한 결과 이 같이 나타났다고 14일 밝혔다 50대 이상의 유튜브 사용시간은 101억분으로 가장 많았다 이는 지난해 4월 51억분에서 두 배가량 늘어난 수치다 다음으로 10대 89억분 20대 81억분 30대 61억분 40대 57억분 등 순이었다 1인 평균 시청 시간은 10대가 평균 1895분 월 31시간 35분 으로 가장 길었다 20대 1625분 가 그 뒤를 이었고 50대 이상은 145분으로 30대 988분 와 40대 781분 보다 높았다 유튜브 앱 사용시간은 총 388억분으로 지난해 4월 258억분보다 50 늘었다 카카오톡 사용시간은 225억분 네이버 153억분 페이스북 42억분 등 순이었다 이들 앱의 사용시간은 지난해보다 각각 19 21 5 성장했지만 유튜브와의 격차는 더욱 벌어졌다 곽희양 기자 오늘의 인기뉴스 김민아 칼럼 황교안적 인 너무도 황교안적 인이낙연 총선 등판 기정사실화하나박해미 음주 사고 황민과 결국 이혼 25년 결혼생활 종지부미중 관세확전에 세계증시 패닉 공포지수 30 급등 단독 경찰의 청와대 보고용 2016년 총선 보고서 본 유명 선거 컨설턴트 우린 억만금 줘도 이런 건 못 만든다 최신 뉴스 두고 두고 읽는 뉴스 인기 무료만화 경향신문 무단전재 및 재배포 금지 1 8 1 8 

문서이름:113 / 거리:4.1545
 총 이용시간 258억 분에서 388억 분 50 키뉴스 이길주 기자 앱 리테일 분석서비스 와이즈앱이 지난 4월 한국 안드로이드 스마트폰 사용자의 세대별 사용 현황을 발표했다 전 세대를 합쳐 한국인이 가장 오래 사용한 앱은 유튜브로 4월 한달 총 사용시간 388억 분을 이용했다 그 뒤를 카카오톡 225억 분 네이버 153억 분 페이스북 42억 분의 순이었다 한국인이 오래 사용하는

## Precesion(정확율) : 검색결과는 최대한 많이,
## Recall(재현율) : 유사도는 최대한 높이
 